# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [63]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from geoapify_key import geoapify_key

In [64]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,8.874,66,92,14.46,TF,1706483964
1,1,hobyo,5.3505,48.5268,46.908,82,70,4.60,SO,1706483964
2,2,tagiura,32.8817,13.3506,26.784,72,9,5.42,LY,1706483964
3,3,asadabad,34.7824,48.1185,-1.854,87,56,1.74,IR,1706483964
4,4,lompoc,34.6391,-120.4579,48.744,33,0,4.63,US,1706483965


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [65]:
#create the dictionary to hold the coordinates
coordinates = {
    "latitude": city_data_df['Lat'],
    "longitude":city_data_df["Lng"],
    "City": city_data_df["City"],
    "Humidity":city_data_df["Humidity"]  
}

coordinates_df = pd.DataFrame(coordinates)

In [66]:
%%capture --no-display

# Configure the map plot
map_plot_1 = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 580,
    size = "Humidity",
    scale =1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [75]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.copy()

#filter for ideal conditions max temp <27, max temp >21,wind speed <4.5 m/s,cloudiness = 0
new_df = pd.DataFrame(ideal_df[(ideal_df['Max Temp'] < 75) & (ideal_df['Max Temp'] > 45) & \
                              (ideal_df['Wind Speed'] < 4.5) & (ideal_df['Cloudiness'] ==0)])

#Drop any rows with null values
new_df.dropna(inplace=True)

#display 
new_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,salinas,36.6777,-121.6555,46.890,39,0,2.06,US,1706483968
70,70,quelimane,-17.8786,36.8883,46.746,78,0,0.00,MZ,1706483976
179,179,nuriootpa,-34.4833,139.0000,46.656,29,0,2.83,AU,1706484002
183,183,kintampo,8.0563,-1.7306,49.068,17,0,0.81,GH,1706484003
242,242,leeton,-34.5667,146.4000,54.900,35,0,1.65,AU,1706484023


### Step 3: Create a new DataFrame called `hotel_df`.

In [76]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(new_df[['City','Country','Lat','Lng','Humidity']])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
24,salinas,US,36.6777,-121.6555,39,
70,quelimane,MZ,-17.8786,36.8883,78,
179,nuriootpa,AU,-34.4833,139.0000,29,
183,kintampo,GH,8.0563,-1.7306,17,
242,leeton,AU,-34.5667,146.4000,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [77]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
   
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    categories=f"categories=accommodation.hotel&"
    rad = f"filter=circle:{longitude},{latitude},{radius}&"
    bias = f"bias=proximity:{longitude},{latitude}&"
    limit = "limit=20&"
    apiKey = f"apiKey={geoapify_key}"
    
    #end point for the url
    together = categories + rad + limit + apiKey
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request
    response = requests.get(base_url, together)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
salinas - nearest hotel: The Traveler's Hotel
quelimane - nearest hotel: Hotel Chuabo
nuriootpa - nearest hotel: The Vine Inn
kintampo - nearest hotel: Yakam Hotel
leeton - nearest hotel: Leeton Hotel
cabo san lucas - nearest hotel: Hotel Riu Santa Fe
lagunillas - nearest hotel: Hotel América


,City,Country,Lat,Lng,Humidity,Hotel Name
24,salinas,US,36.6777,-121.6555,39,The Traveler's Hotel
70,quelimane,MZ,-17.8786,36.8883,78,Hotel Chuabo
179,nuriootpa,AU,-34.4833,139.0000,29,The Vine Inn
183,kintampo,GH,8.0563,-1.7306,17,Yakam Hotel
242,leeton,AU,-34.5667,146.4000,35,Leeton Hotel
415,cabo san lucas,MX,22.8909,-109.9124,52,Hotel Riu Santa Fe
567,lagunillas,VE,10.1301,-71.2595,60,Hotel América


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [78]:
%%capture --no-display

#create a dictionary 
coordinates1 = {
    "latitude": hotel_df['Lat'],
    "longitude":hotel_df["Lng"],
    "City": hotel_df["City"],
    "Humidity":hotel_df["Humidity"],  
    "Hotel Name": hotel_df["Hotel Name"],
    "Country": hotel_df["Country"]
}

#convert the dictionary to a dataframe that will be plotted
coordinates1_df = pd.DataFrame(coordinates1)

# Configure the map plot
map_plot_2 = coordinates1_df.hvplot.points(
    "longitude",
    "latitude",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 580,
    size = "Humidity",
    scale =1,
    color = "City",
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity,Hotel Name,Country)